#Imports

In [ ]:
%pip install langchain==0.0.268 chromadb==0.3.26 sentence_transformers==2.2.2 InstructorEmbedding==1.0.1 fastapi pydantic==1.10.13 pyspark

In [ ]:
!python3 --version
#3.10.12

Python 3.10.12


In [ ]:
%run config.py

# Functions.py

In [ ]:
%run functions.py

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:88: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


.gitattributes:   0%|          | 0.00/1.18k [00:00<?, ?B/s]

1_Pooling/config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/10.7k [00:00<?, ?B/s]

config.json:   0%|          | 0.00/612 [00:00<?, ?B/s]

...

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/350 [00:00<?, ?B/s]

train_script.py:   0%|          | 0.00/13.2k [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

# EDA

In [ ]:
#Problem statement - Predicting whether a specific customer is likely to subscribe to a term deposit

In [ ]:
from pyspark.sql import SparkSession

# Create a Spark session
spark = SparkSession.builder.appName("Banking_look_alike_modelling").getOrCreate()


In [ ]:
input_df = spark.read.option("header", "true").option("inferSchema", "true").csv("./bank.csv")

In [ ]:
from pyspark.sql.functions import monotonically_increasing_id

input_df= input_df.withColumn("id", monotonically_increasing_id())


input_df.show()
#input_df.select("id").show()


+---+-----------+--------+---------+-------+-------+-------+----+-------+---+-----+--------+--------+-----+--------+--------+-------+---+
|age|        job| marital|education|default|balance|housing|loan|contact|day|month|duration|campaign|pdays|previous|poutcome|deposit| id|
+---+-----------+--------+---------+-------+-------+-------+----+-------+---+-----+--------+--------+-----+--------+--------+-------+---+
| 59|     admin.| married|secondary|     no|   2343|    yes|  no|unknown|  5|  may|    1042|       1|   -1|       0| unknown|    yes|  0|
| 56|     admin.| married|secondary|     no|     45|     no|  no|unknown|  5|  may|    1467|       1|   -1|       0| unknown|    yes|  1|
| 41| technician| married|secondary|     no|   1270|    yes|  no|unknown|  5|  may|    1389|       1|   -1|       0| unknown|    yes|  2|
| 55|   services| married|secondary|     no|   2476|    yes|  no|unknown|  5|  may|     579|       1|   -1|       0| unknown|    yes|  3|
| 54|     admin.| married| tertiar

In [ ]:
train_df, test_df = input_df.randomSplit([0.8, 0.2], seed=42)

In [ ]:
test_df.count()

2164

In [ ]:
input_df.printSchema()

root
 |-- age: integer (nullable = true)
 |-- job: string (nullable = true)
 |-- marital: string (nullable = true)
 |-- education: string (nullable = true)
 |-- default: string (nullable = true)
 |-- balance: integer (nullable = true)
 |-- housing: string (nullable = true)
 |-- loan: string (nullable = true)
 |-- contact: string (nullable = true)
 |-- day: integer (nullable = true)
 |-- month: string (nullable = true)
 |-- duration: integer (nullable = true)
 |-- campaign: integer (nullable = true)
 |-- pdays: integer (nullable = true)
 |-- previous: integer (nullable = true)
 |-- poutcome: string (nullable = true)
 |-- deposit: string (nullable = true)
 |-- id: long (nullable = false)



In [ ]:
input_df.show(20)
input_df.groupBy("deposit").count().show()

+---+-----------+--------+---------+-------+-------+-------+----+-------+---+-----+--------+--------+-----+--------+--------+-------+---+
|age|        job| marital|education|default|balance|housing|loan|contact|day|month|duration|campaign|pdays|previous|poutcome|deposit| id|
+---+-----------+--------+---------+-------+-------+-------+----+-------+---+-----+--------+--------+-----+--------+--------+-------+---+
| 59|     admin.| married|secondary|     no|   2343|    yes|  no|unknown|  5|  may|    1042|       1|   -1|       0| unknown|    yes|  0|
| 56|     admin.| married|secondary|     no|     45|     no|  no|unknown|  5|  may|    1467|       1|   -1|       0| unknown|    yes|  1|
| 41| technician| married|secondary|     no|   1270|    yes|  no|unknown|  5|  may|    1389|       1|   -1|       0| unknown|    yes|  2|
| 55|   services| married|secondary|     no|   2476|    yes|  no|unknown|  5|  may|     579|       1|   -1|       0| unknown|    yes|  3|
| 54|     admin.| married| tertiar

In [ ]:
# Create row containing all columns in the format (column_name: column_value) except target
rows_to_convert = input_df.columns
rows_to_convert.remove("deposit")
rows_to_convert

['age',
 'job',
 'marital',
 'education',
 'default',
 'balance',
 'housing',
 'loan',
 'contact',
 'day',
 'month',
 'duration',
 'campaign',
 'pdays',
 'previous',
 'poutcome',
 'id']

In [ ]:
train_df = get_row_as_text(input_df, rows_to_convert)
train_df.printSchema()

root
 |-- age: integer (nullable = true)
 |-- job: string (nullable = true)
 |-- marital: string (nullable = true)
 |-- education: string (nullable = true)
 |-- default: string (nullable = true)
 |-- balance: integer (nullable = true)
 |-- housing: string (nullable = true)
 |-- loan: string (nullable = true)
 |-- contact: string (nullable = true)
 |-- day: integer (nullable = true)
 |-- month: string (nullable = true)
 |-- duration: integer (nullable = true)
 |-- campaign: integer (nullable = true)
 |-- pdays: integer (nullable = true)
 |-- previous: integer (nullable = true)
 |-- poutcome: string (nullable = true)
 |-- deposit: string (nullable = true)
 |-- id: long (nullable = false)
 |-- row_as_text: string (nullable = false)



In [ ]:
test_df.printSchema()

root
 |-- age: integer (nullable = true)
 |-- job: string (nullable = true)
 |-- marital: string (nullable = true)
 |-- education: string (nullable = true)
 |-- default: string (nullable = true)
 |-- balance: integer (nullable = true)
 |-- housing: string (nullable = true)
 |-- loan: string (nullable = true)
 |-- contact: string (nullable = true)
 |-- day: integer (nullable = true)
 |-- month: string (nullable = true)
 |-- duration: integer (nullable = true)
 |-- campaign: integer (nullable = true)
 |-- pdays: integer (nullable = true)
 |-- previous: integer (nullable = true)
 |-- poutcome: string (nullable = true)
 |-- deposit: string (nullable = true)
 |-- id: long (nullable = false)



In [ ]:
train_df.select("row_as_text").show(10, truncate=False)

+-----------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------+
|row_as_text                                                                                                                                                                                                                              |
+-----------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------+
|age: 59; job: admin.; marital: married; education: secondary; default: no; balance: 2343; housing: yes; loan: no; contact: unknown; day: 5; month: may; duration: 1042; campaign: 1; pdays: -1; previous: 0; poutcome: unknown; id: 0    |
|age: 56; job: admin.; marital: married; education: seco

# Create Retriever

In [ ]:

import os

# Specify the path of the directory you want to create
db_dir = "/content/embeddings"

# Create the directory
os.makedirs(db_dir, exist_ok=True)

In [ ]:
texts_list = train_df.rdd.collect()
texts_list[0]

Row(age=59, job='admin.', marital='married', education='secondary', default='no', balance=2343, housing='yes', loan='no', contact='unknown', day=5, month='may', duration=1042, campaign=1, pdays=-1, previous=0, poutcome='unknown', deposit='yes', id=0, row_as_text='age: 59; job: admin.; marital: married; education: secondary; default: no; balance: 2343; housing: yes; loan: no; contact: unknown; day: 5; month: may; duration: 1042; campaign: 1; pdays: -1; previous: 0; poutcome: unknown; id: 0')

In [ ]:

train_df.count()

11162

In [ ]:

step = 500
k = 2000
hf_embeddings = get_embedding_model()
# texts_label_list = train_df.rdd.map(lambda x: (x.row_as_text, x.deposit)).collect()
# # texts_list = [x[0] for x in texts_label_list]
# # texts_list
# vdb = Chroma(persist_directory=db_dir, embedding_function=hf_embeddings, collection_metadata={"hnsw:space": "cosine"})
# for i in range(0, len(texts_label_list), step):
#     texts = [x[0] for x in texts_label_list[i:i+step]]
#     metadata = [{"label": x[1]} for x in texts_label_list[i:i+step]]
#     vdb.add_texts(texts, metadata)
# vdb.persist()
# retriever = vdb.as_retriever(search_kwargs={"k": k})


In [ ]:
vdb = Chroma(persist_directory=db_dir, embedding_function=hf_embeddings,)
for i in range(0, len(texts_list), step):
    texts = [x.row_as_text for x in texts_list[i:i+step]]
    metadata = [{"id": str(x.id), "age": str(x.age), "job": str(x.job), "marital": str(x.marital), "education": str(x.education), "default": str(x.default), "balance": str(x.balance), "housing": str(x.housing), "loan": str(x.loan), "contact": str(x.contact), "day": str(x.day), "deposit": str(x.deposit), "duration": str(x.duration), "campaign": str(x.campaign), "pdays": str(x.pdays),} for x in texts_list[i:i+step]]
    vdb.add_texts(texts, metadata)
vdb.persist()

In [ ]:

from google.colab import files
import shutil

# Path to the directory you want to download
directory_to_zip = '/content/embeddings'

# Create a zip file of the directory
shutil.make_archive('/content/embeddings', 'zip', directory_to_zip)

# Download the zip file
files.download('/content/embeddings.zip')


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [ ]:
retriever = vdb.as_retriever(search_kwargs={"k": 50})

# Validataion per label

In [ ]:
val_A, _ = train_df.filter(F.col("deposit") == "yes").randomSplit([0.20, 0.80])
val_B, _ = train_df.filter(F.col("deposit") == "no").randomSplit([0.20, 0.80])


In [ ]:
val_A_row2txt = get_row_as_text(val_A.limit(10), rows_to_convert)
val_B_row2txt = get_row_as_text(val_B.limit(10), rows_to_convert)


In [ ]:
hf_embeddings = get_embedding_model()  # Define get_embedding_model() based on your requirements
print(f"\n\nNumber of embeddings in Chroma: {vdb._collection.count()}")
print(f"\n\nNumber of rows in dataframe: {train_df.count()}")



Number of embeddings in Chroma: 11162


Number of rows in dataframe: 11162


In [ ]:
def get_retrived_df(input_df, val_df, retriever):
  texts_list = val_df.rdd.map(lambda x: x.row_as_text).collect()

  input_rows = val_df.rdd.map(lambda x: x.row_as_text).collect()
  relevant_rows = []

  step = 5
  for i in range(0, len(input_rows)):
      for relevant_row in retriever.get_relevant_documents("\n".join(input_rows[i])):
          relevant_rows.append(relevant_row.page_content + f"; predicted: {relevant_row.metadata['deposit']}; id: {relevant_row.metadata['id']}")

  converted_rows = [dict(pair.split(": ") for pair in row.split("; ")) for row in relevant_rows]
  return spark.createDataFrame(converted_rows).distinct().join(input_df.select("id", "deposit"), how="left", on=["id"])


In [ ]:
get_retrived_df(input_df, val_A, retriever).groupBy("deposit").count().show()

+-------+-----+
|deposit|count|
+-------+-----+
|     no|   83|
|    yes|  225|
+-------+-----+



In [ ]:
get_retrived_df(input_df, val_B, retriever).groupBy("deposit").count().show()

+-------+-----+
|deposit|count|
+-------+-----+
|     no|   97|
|    yes|  233|
+-------+-----+



In [ ]:
input_rows = val_A_row2txt.rdd.map(lambda x: x.row_as_text).collect()
relevant_rows = []
retrieved_labels = []
for i in range(0, len(input_rows), 500):
    for relevant_row in retriever.get_relevant_documents("\n".join(input_rows[i:(i+500)])):
        relevant_rows.append(relevant_row.page_content)
        retrieved_labels.append(relevant_row.metadata["deposit"])

converted_rows = [dict(pair.split(": ") for pair in row.split("; ")) for row in relevant_rows]
retrieved_df = spark.createDataFrame(converted_rows)

In [ ]:
result_df_A = get_retrived_df(input_df, val_A_row2txt, retriever)
result_df_A.distinct().groupBy("deposit").count().show()

+-------+-----+
|deposit|count|
+-------+-----+
|     no|   20|
|    yes|   85|
+-------+-----+



In [ ]:
get_retrived_df(input_df, val_B_row2txt, retriever).groupBy("deposit").count().show()

+-------+-----+
|deposit|count|
+-------+-----+
|     no|   30|
|    yes|   94|
+-------+-----+



In [ ]:
test_df = get_row_as_text(test_df, rows_to_convert).limit(1000)

In [ ]:
from collections import Counter


def get_retrieved_rows(input_row, retriever):
  relevant_rows = []
  labels = []
  for relevant_row in retriever.get_relevant_documents(input_row):
      relevant_rows.append(relevant_row.page_content + f"; predicted: {relevant_row.metadata['deposit']}; id: {relevant_row.metadata['id']}")
      labels.append(relevant_row.metadata['deposit'])


  converted_rows = [dict(pair.split(": ") for pair in row.split("; ")) for row in relevant_rows]
  spark.createDataFrame(converted_rows).distinct().join(input_df.select("id", "deposit"), how="left", on=["id"]).groupBy("deposit").count().show()

  # most common label
  counter = Counter(labels)
  most_common_label, count = counter.most_common(1)[0]
  return most_common_label

from pyspark.sql.functions import udf
from pyspark.sql.types import StringType

get_retrieved_rows_udf = udf(lambda x: get_retrieved_rows(x, retriever), StringType())


In [ ]:
import pandas as pd
test_pdf = test_df.toPandas()
test_pdf["predicted"] = test_pdf["row_as_text"].apply(lambda x: get_retrieved_rows(x, retriever))

Streaming output truncated to the last 5000 lines.
+-------+-----+

+-------+-----+
|deposit|count|
+-------+-----+
|     no|   29|
|    yes|   21|
+-------+-----+

+-------+-----+
|deposit|count|
+-------+-----+
|     no|   34|
|    yes|   16|
+-------+-----+

+-------+-----+
|deposit|count|
+-------+-----+
|     no|   24|
|    yes|   26|
+-------+-----+

+-------+-----+
|deposit|count|
+-------+-----+
|     no|   32|
|    yes|   18|
+-------+-----+

+-------+-----+
|deposit|count|
+-------+-----+
|     no|   36|
|    yes|   14|
+-------+-----+

+-------+-----+
|deposit|count|
+-------+-----+
|     no|   37|
|    yes|   13|
+-------+-----+

+-------+-----+
|deposit|count|
+-------+-----+
|     no|   40|
|    yes|   10|
+-------+-----+

+-------+-----+
|deposit|count|
+-------+-----+
|     no|   42|
|    yes|    8|
+-------+-----+

+-------+-----+
|deposit|count|
+-------+-----+
|     no|   25|
|    yes|   25|
+-------+-----+

+-------+-----+
|deposit|count|
+-------+-----+
|     no|  

In [ ]:
# #retrieving relevant documents, extracting information from them, converting the data into a specific format
# from collections import Counter

# def get_retrieved_rows(input_row, retriever):
#   relevant_rows = []
#   labels = []
#   for relevant_row in retriever.get_relevant_documents(input_row):
#       relevant_rows.append(relevant_row.page_content)
#       labels.append(relevant_row.metadata["label"])

#   converted_rows = [dict(pair.split(": ") for pair in row.split("; ")) for row in relevant_rows]
#   spark.createDataFrame(converted_rows).distinct().join(input_df.select("ID", "deposit"), how="left", on=["ID"]).groupBy("deposit").count().show()

#   # most common label
#   counter = Counter(labels)
#   most_common_label, count = counter.most_common(1)[0]
#   return most_common_label

# from pyspark.sql.functions import udf
# from pyspark.sql.types import StringType

# get_retrieved_rows_udf = udf(lambda x: get_retrieved_rows(x, retriever), StringType())


# Validataion on test data

In [ ]:
test_df.show()

In [ ]:
test_df.count()

In [ ]:
#sample_df =sample_df.drop("deposit")
sample_df = get_row_as_text(test_df, rows_to_convert)
sample_df =sample_df.drop("deposit")
#sample_df =sample_df.drop("deposit")
sample_df.count()



In [ ]:
sample_list = sample_df.rdd.map(lambda x: x.row_as_text).collect()

In [ ]:
#determine the most common label among the relevant documents retrieved by a retriever for a given input row.


from collections import Counter


def get_retrieved_rows(input_row, retriever):
  relevant_rows = []
  labels = []
  for relevant_row in retriever.get_relevant_documents(input_row):
      relevant_rows.append(relevant_row.page_content + f"; predicted: {relevant_row.metadata['deposit']}; id: {relevant_row.metadata['id']}")
      labels.append(relevant_row.metadata['deposit'])


  converted_rows = [dict(pair.split(": ") for pair in row.split("; ")) for row in relevant_rows]
  spark.createDataFrame(converted_rows).distinct().join(input_df.select("id", "deposit"), how="left", on=["id"]).groupBy("deposit").count().show()

  # most common label
  counter = Counter(labels)
  most_common_label, count = counter.most_common(1)[0]
  return most_common_label

from pyspark.sql.functions import udf
from pyspark.sql.types import StringType

get_retrieved_rows_udf = udf(lambda x: get_retrieved_rows(x, retriever), StringType())



In [ ]:
sample_pdf = sample_df.toPandas()
sample_pdf["predicted"] = sample_pdf["row_as_text"].apply(lambda x: get_retrieved_rows(x, retriever))

Streaming output truncated to the last 5000 lines.
+-------+-----+

+-------+-----+
|deposit|count|
+-------+-----+
|     no|   27|
|    yes|   23|
+-------+-----+

+-------+-----+
|deposit|count|
+-------+-----+
|     no|   27|
|    yes|   23|
+-------+-----+

+-------+-----+
|deposit|count|
+-------+-----+
|     no|   32|
|    yes|   18|
+-------+-----+

+-------+-----+
|deposit|count|
+-------+-----+
|     no|   32|
|    yes|   18|
+-------+-----+

+-------+-----+
|deposit|count|
+-------+-----+
|     no|   33|
|    yes|   17|
+-------+-----+

+-------+-----+
|deposit|count|
+-------+-----+
|     no|   33|
|    yes|   17|
+-------+-----+

+-------+-----+
|deposit|count|
+-------+-----+
|     no|   30|
|    yes|   20|
+-------+-----+

+-------+-----+
|deposit|count|
+-------+-----+
|     no|   30|
|    yes|   20|
+-------+-----+

+-------+-----+
|deposit|count|
+-------+-----+
|     no|   31|
|    yes|   19|
+-------+-----+

+-------+-----+
|deposit|count|
+-------+-----+
|     no|  

In [ ]:
sample_pdf.count()

age            1000
job            1000
marital        1000
education      1000
default        1000
balance        1000
housing        1000
loan           1000
contact        1000
day            1000
month          1000
duration       1000
campaign       1000
pdays          1000
previous       1000
poutcome       1000
id             1000
row_as_text    1000
predicted      1000
dtype: int64

In [ ]:
result_dff = spark.createDataFrame(sample_pdf)

In [ ]:
result_dff.select("id","predicted").show()

In [ ]:
test_df.select("deposit","id").show()

In [ ]:
# Selecting the relevant columns from each dataframe
from pyspark.sql.functions import col, when
selected_test_df = test_df.select("id", "deposit")
selected_result_df = result_dff.select("id", "predicted")

# Joining the two dataframes on 'id'
joined_df = selected_test_df.join(selected_result_df, on="id")

# Comparing the 'deposit' and 'predicted' columns
comparison_result = joined_df.withColumn("comparison", col("deposit") == col("predicted"))

# Displaying the result
comparison_result.show()


In [ ]:
comparison_result = joined_df.withColumn("comparison", when(col("deposit") == col("predicted"), "same").otherwise("different"))


count_result = comparison_result.groupBy("comparison").count()

count_result.show()

Using cross encoder -> compare two peice of text and undertsand the context and score similarity




In [ ]:
# from helper_utils import load_chroma, word_wrap, project_embeddings
from chromadb.utils.embedding_functions import SentenceTransformerEmbeddingFunction
import numpy as np

In [ ]:
from sentence_transformers import CrossEncoder
cross_encoder = CrossEncoder('cross-encoder/ms-marco-MiniLM-L-6-v2')

In [ ]:
from collections import Counter

#Python function designed to retrieve relevant rows using some form of retriever, apply a cross-encoder model to them, and return the most common label among the retrieved rows.
def get_retrieved_rows_with_crossenc(input_row, retriever, k):
    relevant_rows = []
    labels = []
    for relevant_row in retriever.get_relevant_documents(input_row):
      score = cross_encoder.predict([input_row, relevant_row.page_content])
      #score represents the relevance of the input_row and the retrieved row.
      relevant_rows.append(relevant_row.page_content + f"; predicted: {relevant_row.metadata['deposit']}; id: {relevant_row.metadata['id']}; Score: {score}")
      labels.append(relevant_row.metadata['deposit'])


    converted_rows = [dict(pair.split(": ") for pair in row.split("; ")) for row in relevant_rows]
    df = spark.createDataFrame(converted_rows).distinct().join(input_df.select("id", "deposit"), how="left", on=["id"])
    # df.groupBy("Segmentation").count().show()

    df = spark.createDataFrame(converted_rows).distinct().join(input_df.select("id", "deposit"), how="left", on=["id"])
    most_common_label = df.orderBy(df["Score"].desc()).limit(int(k*0.1)).groupBy("deposit").count().orderBy(F.col("count").desc()).first()["deposit"]

    return most_common_label


In [ ]:
k = 100
retriever = vdb.as_retriever(search_kwargs={"k": k})

In [ ]:
sample_df.count()

1000

In [ ]:
import pandas as pd
#test_df = spark.read.option("header", "true").option("inferSchema", "true").csv("resources/data/Test.csv")
test_df = get_row_as_text(test_df, rows_to_convert).limit(100)

test_pdf = test_df.toPandas()
test_pdf["predicted"] = test_pdf["row_as_text"].apply(lambda x: get_retrieved_rows_with_crossenc(x, retriever, k))

In [ ]:
# sample_pdff = sample_df.toPandas()
# sample_pdff["predicted"] = sample_pdff["row_as_text"].apply(lambda x: get_retrieved_rows_with_crossenc(x, retriever,k))

In [ ]:
result_pdff = spark.createDataFrame(test_pdf)

In [ ]:
# Selecting the relevant columns from each dataframe
from pyspark.sql.functions import col, when
selected_test_df = test_df.select("id", "deposit")
selected_result_df = result_pdff.select("id", "predicted")

# Joining the two dataframes on 'id'
joined_df = selected_test_df.join(selected_result_df, on="id")

# Comparing the 'deposit' and 'predicted' columns
comparison_result = joined_df.withColumn("comparison", col("deposit") == col("predicted"))

# Displaying the result
comparison_result.show()


+----+-------+---------+----------+
|  id|deposit|predicted|comparison|
+----+-------+---------+----------+
|6456|     no|      yes|     false|
|2464|    yes|      yes|      true|
|2406|    yes|      yes|      true|
|2798|    yes|      yes|      true|
|4291|    yes|      yes|      true|
|4444|    yes|      yes|      true|
|4552|    yes|      yes|      true|
|7327|     no|       no|      true|
|8538|     no|      yes|     false|
|3837|    yes|      yes|      true|
|8675|     no|      yes|     false|
|1430|    yes|      yes|      true|
|2861|    yes|      yes|      true|
|3217|    yes|      yes|      true|
|4249|    yes|      yes|      true|
|4718|    yes|      yes|      true|
|2297|    yes|      yes|      true|
|1233|    yes|      yes|      true|
|2348|    yes|      yes|      true|
|8620|     no|      yes|     false|
+----+-------+---------+----------+
only showing top 20 rows



In [ ]:
comparison_result = joined_df.withColumn("comparison", when(col("deposit") == col("predicted"), "same").otherwise("different"))


count_result = comparison_result.groupBy("comparison").count()

count_result.show()

+----------+-----+
|comparison|count|
+----------+-----+
|      same|   79|
| different|   21|
+----------+-----+



In [ ]:
#  Calculate precision, recall, and F1-score

In [ ]:

tp = joined_df.filter((col("deposit") == "yes") & (col("predicted") == "yes")).count()
fp = joined_df.filter((col("deposit") == "no") & (col("predicted") == "yes")).count()
tn = joined_df.filter((col("deposit") == "no") & (col("predicted") == "no")).count()
fn = joined_df.filter((col("deposit") == "yes") & (col("predicted") == "no")).count()


precision = tp / (tp + fp)
recall = tp / (tp + fn)
f1_score = 2 * (precision * recall) / (precision + recall)


In [ ]:
print(precision)        #Of all the instances predicted as positive, how many are actually positive?"

0.8356164383561644


In [ ]:
print(recall)      #Of all the actual positive instances, how many did the model correctly predict as positive?"



0.8714285714285714


In [ ]:
print(f1_score)      #harmonic mean of precision and recall,

0.8531468531468531
